In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.models import load_model

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Using TensorFlow backend.


In [2]:
#Load Model

#resnet50-noaug_no-normals
print('loading resnet50-noaug_no-normals')
model = 'models/classification_models/resnet_50-noaug_no-normals'
model_name = 'weights-improvement-19-0.78.hdf5'          
model_dir = os.path.join(model,model_name)
resnet50_noaug_no_normals = load_model(model_dir)

#resnet50-aug_no-normals
print('loading resnet50-aug_no-normals')
model = 'models/classification_models/resnet_50-aug_no-normals'
model_name = 'weights-improvement-18-0.78.hdf5'          
model_dir = os.path.join(model,model_name)
resnet50_aug_no_normals = load_model(model_dir)

#inception_resnet_v2-noaug_no-normals
print('loading inception_resnet_v2-noaug_no-normals')
model = 'models/classification_models/inception_resnet_v2-noaug_no-normals'
model_name = 'weights-improvement-27-0.89.hdf5'          
model_dir = os.path.join(model,model_name)
inception_resnet_v2_noaug_no_normals = load_model(model_dir)

#inception_resnet_v2-aug_no-normals
print('loading inception_resnet_v2-aug_no-normals')
model = 'models/classification_models/inception_resnet_v2-aug_no-normals'
model_name = 'weights-improvement-32-0.89.hdf5'          
model_dir = os.path.join(model,model_name)
inception_resnet_v2_aug_no_normals = load_model(model_dir)

#inception_resnet_v2-noaug_no-normals_clahe_eq
print('loading inception_resnet_v2-noaug_no-normals_clahe_eq')
model = 'models/classification_models/inception_resnet_v2-noaug_no-normals_clahe_eq'
model_name = 'weights-improvement-28-0.81.hdf5'          
model_dir = os.path.join(model,model_name)
inception_resnet_v2_noaug_no_normals_clahe_eq = load_model(model_dir)

#inception_resnet_v2-aug_no-normals_clahe_eq
print('loading inception_resnet_v2-aug_no-normals_clahe_eq')
model = 'models/classification_models/inception_resnet_v2-aug_no-normals_clahe_eq'
model_name = 'weights-improvement-41-0.73.hdf5'          
model_dir = os.path.join(model,model_name)
inception_resnet_v2_aug_no_normals_clahe_eq = load_model(model_dir)

loading resnet50-noaug_no-normals
loading resnet50-aug_no-normals
loading inception_resnet_v2-noaug_no-normals
loading inception_resnet_v2-aug_no-normals
loading inception_resnet_v2-noaug_no-normals_clahe_eq
loading inception_resnet_v2-noaug_no-normals_clahe_eq


In [3]:
train_set = pd.read_pickle('train_set_class_nn.pickle')
val_set = pd.read_pickle('val_set_class_nn.pickle')

In [4]:
val_set.head()

,image_path,images,lesion_state,lesion_state_one_hot
474,/external_drive/BUS_Deep_Learning/data/Dataset...,benign (345).png,benign,"[1, 0]"
703,/external_drive/BUS_Deep_Learning/data/Dataset...,malignant (170).png,malignant,"[0, 1]"
278,/external_drive/BUS_Deep_Learning/data/Dataset...,benign (278).png,benign,"[1, 0]"
299,/external_drive/BUS_Deep_Learning/data/Dataset...,benign (304).png,benign,"[1, 0]"
632,/external_drive/BUS_Deep_Learning/data/Dataset...,malignant (61).png,malignant,"[0, 1]"


In [5]:
classes = ['benign','malignant']
gts = []
preds_resnet50_noaug_no_normals = [] 
preds_resnet50_aug_no_normals = [] 
preds_inception_resnet_v2_noaug_no_normals = [] 
preds_inception_resnet_v2_aug_no_normals = [] 
preds_inception_resnet_v2_noaug_no_normals_clahe_eq = [] 
preds_inception_resnet_v2_aug_no_normals_clahe_eq = [] 

for index, row in val_set.iterrows():
    print('loading sequence : {}'.format(row['image_path']))
    img = cv2.imread(row['image_path'], cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(128,128))
    img = img/255.0
    img = np.array([img.reshape(img.shape+(1,))])
    pred_resnet50_noaug_no_normals = resnet50_noaug_no_normals.predict(img)
#     print(pred_resnet50_noaug_no_normals)
    pred_resnet50_aug_no_normals = resnet50_aug_no_normals.predict(img)
#     print(pred_resnet50_aug_no_normals)
    pred_inception_resnet_v2_noaug_no_normals = inception_resnet_v2_noaug_no_normals.predict(img)
#     print(pred_inception_resnet_v2_noaug_no_normals)
    pred_inception_resnet_v2_aug_no_normals = inception_resnet_v2_aug_no_normals.predict(img)
#     print(pred_inception_resnet_v2_aug_no_normals)
    pred_inception_resnet_v2_noaug_no_normals_clahe_eq = inception_resnet_v2_noaug_no_normals_clahe_eq.predict(img)    
#     print(pred_inception_resnet_v2_noaug_no_normals_clahe_eq)
    pred_inception_resnet_v2_aug_no_normals_clahe_eq = inception_resnet_v2_aug_no_normals_clahe_eq.predict(img)    
#     print(pred_inception_resnet_v2_aug_no_normals_clahe_eq)

    preds_resnet50_noaug_no_normals.append(classes[np.where(pred_resnet50_noaug_no_normals[0]==max(pred_resnet50_noaug_no_normals[0]))[0][0]])
    preds_resnet50_aug_no_normals.append(classes[np.where(pred_resnet50_aug_no_normals[0]==max(pred_resnet50_aug_no_normals[0]))[0][0]])
    preds_inception_resnet_v2_noaug_no_normals.append(classes[np.where(pred_inception_resnet_v2_noaug_no_normals[0]==max(pred_inception_resnet_v2_noaug_no_normals[0]))[0][0]])
    preds_inception_resnet_v2_aug_no_normals.append(classes[np.where(pred_inception_resnet_v2_aug_no_normals[0]==max(pred_inception_resnet_v2_aug_no_normals[0]))[0][0]])
    preds_inception_resnet_v2_noaug_no_normals_clahe_eq.append(classes[np.where(pred_inception_resnet_v2_noaug_no_normals_clahe_eq[0]==max(pred_inception_resnet_v2_noaug_no_normals_clahe_eq[0]))[0][0]])
    preds_inception_resnet_v2_aug_no_normals_clahe_eq.append(classes[np.where(pred_inception_resnet_v2_aug_no_normals_clahe_eq[0]==max(pred_inception_resnet_v2_aug_no_normals_clahe_eq[0]))[0][0]])

    gts.append(row['lesion_state'])

loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (345).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (170).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (278).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (304).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (61).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (405).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (291).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (33).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (13).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Datase

loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (42).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (412).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (289).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (32).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (301).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (427).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (14).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (191).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (206).png
loading sequence : /external_drive/BUS_Deep_Learning/data/

In [6]:
results = pd.DataFrame(columns=['prediction','ground_truth'])
results['resnet50 + no aug'] = preds_resnet50_noaug_no_normals
results['resnet50 + aug'] = preds_resnet50_aug_no_normals
results['inception_resnet_v2 + no aug'] = preds_inception_resnet_v2_noaug_no_normals
results['inception_resnet_v2 + aug'] = preds_inception_resnet_v2_aug_no_normals
results['inception_resnet_v2 + clahe + no aug'] = preds_inception_resnet_v2_noaug_no_normals_clahe_eq
results['inception_resnet_v2 + clahe + aug'] = preds_inception_resnet_v2_aug_no_normals_clahe_eq
results['ground_truth'] = gts

In [7]:
#accuracy
preds_resnet50_noaug_no_normals_acc = len(results.loc[results['resnet50 + no aug']==results['ground_truth']])/len(results)
preds_resnet50_aug_no_normals_acc = len(results.loc[results['resnet50 + aug']==results['ground_truth']])/len(results)
preds_inception_resnet_v2_noaug_no_normals_acc = len(results.loc[results['inception_resnet_v2 + no aug']==results['ground_truth']])/len(results)
preds_inception_resnet_v2_aug_no_normals_acc = len(results.loc[results['inception_resnet_v2 + aug']==results['ground_truth']])/len(results)
preds_inception_resnet_v2_noaug_no_normals_clahe_eq_acc = len(results.loc[results['inception_resnet_v2 + clahe + no aug']==results['ground_truth']])/len(results)
preds_inception_resnet_v2_aug_no_normals_clahe_eq_acc = len(results.loc[results['inception_resnet_v2 + clahe + aug']==results['ground_truth']])/len(results)

In [10]:
metrics_output = pd.DataFrame(columns = ['model', 'Accuracy'])

metrics_output['model'] = ['resnet50 + no aug', 'resnet50 + aug',
                    'inception_resnet_v2 + no aug', 'inception_resnet_v2 + aug',
                   'inception_resnet_v2 + clahe + no aug', 'inception_resnet_v2 + clahe + aug']

metrics_output['Accuracy'] = [preds_resnet50_noaug_no_normals_acc, preds_resnet50_aug_no_normals_acc,
                       preds_inception_resnet_v2_noaug_no_normals_acc, preds_inception_resnet_v2_aug_no_normals_acc,
                      preds_inception_resnet_v2_noaug_no_normals_clahe_eq_acc, preds_inception_resnet_v2_aug_no_normals_clahe_eq_acc]

metrics_output = metrics_output.set_index('model')

In [11]:
metrics_output

,Accuracy
model,
resnet50 + no aug,0.776923
resnet50 + aug,0.784615
inception_resnet_v2 + no aug,0.884615
inception_resnet_v2 + aug,0.892308
inception_resnet_v2 + clahe + no aug,0.530769
inception_resnet_v2 + clahe + aug,0.676923
